In [2]:
!pip install ultralytics


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.155 🚀 Python-3.10.13 torch-2.7.1 CPU (Apple M4)
Setup complete ✅ (10 CPUs, 32.0 GB RAM, 110.7/1858.2 GB disk)


In [3]:
#VIDEO
from ultralytics import YOLO
# Load a model
model = YOLO('/Users/eliu/Desktop/yolo11x.pt')

# Predict with the model -- tracking on VIDEO
results = model.track(source='/Users/eliu/Documents/chicken_TrackEval_test/chicken_1.mp4',
                tracker="bytetrack.yaml",
                line_width=1,
                agnostic_nms=True,
                #save_crop=True,
                # ave_conf=True,
                save_txt=True,
                save=True,
                #save_frames=True,
                conf=0.1,
                iou=0.4,
                half=True,
                device='mps',
                #vid_stride=60,
                #show=True,
                #visualize=True,
                #stream=True,
                imgsz=640
                )


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/306) /Users/eliu/Documents/chicken_TrackEval_test/chicken_1.mp4: 384x640 16 birds, 1 bowl, 180.8ms
video 1/1 (frame 2/306) /Users/eliu/Documents/chicken_TrackEval_test/chicken_1.mp4: 384x640 16 birds, 1 bowl, 52.7ms
video 1/1 (frame 3/306) /Users/eliu/Documents/chicken_TrackEval_test/chicken_1.mp4: 384x640 15 birds, 1 bowl, 37.4ms
video 1/1 (frame 4/306) /Users/eliu/Documents/chicken_TrackEval_test/chicken_1.mp4: 384x640 14 birds, 1 

In [5]:
# CONVERTING YOLO TO MOT CHALLENGE FORMAT
import os
import glob
import cv2 
# paths
yolo_labels_dir = "/Users/eliu/Documents/benchmark_eval/labels/midwater_simple_labels"
video_path = "/Users/eliu/Documents/benchmark_eval/midwater_simple_V4455_20221130T192123Z_prores.mov" # NEED TO CHANGE IF USED LATER
output_mot_file = "/Users/eliu/Documents/benchmark_eval/mot_files/midwater_simple_mot.txt"

# get frame size from videp
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
if not ret:
    raise ValueError("Failed to read video")
img_height, img_width = frame.shape[:2]
cap.release()
print("dimensions", img_height, img_width)

# unnormalize labels
frame_files = sorted(glob.glob(os.path.join(yolo_labels_dir, "*.txt")))
with open(output_mot_file, "w") as out_file:
    for file in frame_files:
        name = os.path.basename(file).split(".")[0]
        frame_idx = int(name.split("_")[-1])
        with open(file, "r") as f:
            for line in f:
                class_id, x_center, y_center, width, height, track_id = map(float, line.strip().split())
                #calculate
                x = (x_center - width / 2) * img_width
                y = (y_center - height / 2) * img_height
                w = width * img_width
                h = height * img_height

                out_file.write(f"{frame_idx}, {int(track_id)}, {x:.2f}, {y:.2f}, {w:.2f}, {h:.2f}, 1, -1, 1.0\n")

dimensions 1080 1920


ValueError: not enough values to unpack (expected 6, got 5)